python:3.9.13

tensorflow:2.18.0

----

每局只取1个样本：慢  

缓慢降温增加探索性：效果不好。奖励只在最后一步才出现，探索性破坏奖励信号？  

更小卷积核：测试效果很差  

q-learning：测试效果很差  

sarsa：测试效果很差  

补充人工信息：效果提升

增加层数：加了两层，大部分时候表现优于aug，但是有交叉。

降低 learning rate: 从 1e-3 降低到 1e-4, 表现提升。

loss 修正: loss_factor = loss_factor = (len(xs_batch) / batch_size) ** 0.5  似无优势，可能训练过程略微平稳一些

----

目前aug(base4基础上，输入数据增加了人工信息) 略优于 base4(补上了base2遗漏的batch normalization 和 dropout)

bigger(增加了两层) 略优于 aug

出牌信息里去掉带牌？可能意义不大

为什么训练过程中表现有起伏呢？降低 learning rate？增加每个轮次的局数？DQN？

如果DQN的话，要怎么做呢？DQN使用 experience buffer 是为了降低样本的相关性。在斗地主中，如果我一次收集很多局的样本，然后把样本打乱，喂给模型，这样就达到了降低样本相关性的目的吧。在这种情况下，policy network 本身就不是每一步都更新的。还有必要区分 policy network 和 target network 吗？

按照DQN的逻辑，我是不是可以把同一批样本反复随机分割成batch喂给模型？可以加快训练速度吗？

之前 q-learning 的效果太差了，完全没信心

----

trainer输出的loss意义有限。如果发牌高度偏向于地主或农民，那么预测就更准，loss就会更小。一轮只有8局，发牌的影响很大。

----

感觉 DouZero 所谓 Deep Monte-Carlo 跟我的逻辑是一样的。打牌，生成一整局游戏的样本，用最终的输赢作为预测目标，训练状态动作价值模型。多线程不熟悉代码不怎么看得懂，似乎一次生成50局游戏，然后 batch size 32

----
因为电脑是8核，之前都是每8局游戏更新模型。地主农民分三个模型之后，表现下降。可能是因为每个模型的样本数量只有之前的1/3。相应地改成每24局游戏更新一次，似乎表现提升，但是训练速度当然慢了很多。

一个问题，把收集到的样本打乱，32个样本一个batch，不满32个的也当成一个batch，是不是有问题？不满32个的应该怎么处理。假如某组只有1个样本，是不是不太好？

----

怎么把douzero模型下载并跑起来……

----

tensorflow 模型 1696257 参数，运行速度 13ms，torch 模型 1457665 参数，运行速度 4ms。另外的我的模型需要的训练数据，组装速度也较慢，主要是组装额外数据(aug加入)消耗了额外的时间。


In [71]:
import numpy as np
import keras as k
import rules
import arena as arn
import bot_aug
import bot_bigger
import bot_scaledloss
import bot_lr
import bot_three
ARENA = arn.ARENA
bot_rd = arn.bot_rd
bot_me = arn.bot_me


In [72]:
import bot_douzero
import importlib
importlib.reload(bot_douzero)
import torch


In [73]:
def ft(N, bots_f):
    s = 0
    for i in range(N):
        cards = rules.CARDS.copy()
        np.random.shuffle(cards)
        
        arena = ARENA(verbos=0, cards=cards.copy())
        arena.registerbot([bots_f[0](), bots_f[1](), bots_f[2]()])
        arena.wholegame()
        s += arena.winner == 0

    return s

# models & bots

In [77]:
# lr2: 191000
# model = k.models.load_model("model_base2/m{}.keras".format(191000))

model0 = k.models.load_model("model_three_3/0/m{}.keras".format(9000))
def bot0_f(verbos=0):
    return bot_three.BOT(model0, verbos=verbos)

model1 = k.models.load_model("model_three_3/1/m{}.keras".format(9000))
def bot1_f(verbos=0):
    return bot_three.BOT(model1, verbos=verbos)

model2 = k.models.load_model("model_three_3/2/m{}.keras".format(9000))
def bot2_f(verbos=0):
    return bot_three.BOT(model2, verbos=verbos)

# # 79750
# model3 = k.models.load_model("model_scaledloss2/m{}.keras".format(9000))
# def bot3_f(verbos=0):
#     return bot_scaledloss.BOT(model3, verbos=verbos)

model3 = k.models.load_model("e:/ddz_2025_2_model/model_aug/m{}.keras".format(488750))
def bot3_f(verbos=0):
    return bot_aug.BOT(model3, verbos=verbos)

# model3 = k.models.load_model("model_lr2/m{}.keras".format(550))
# def bot3_f(verbos=0):
#     return bot_lr.BOT(model3, verbos=verbos)

def bot_rd_f():
    return bot_rd

def bot_douzero_f(verbos=0):
    return bot_douzero.BOT(verbos)

In [78]:
N = 100
# ft(N, [bot0_f, bot3_f, bot3_f]), ft(N, [bot3_f, bot1_f, bot2_f])
ft(N, [bot3_f, bot_douzero_f, bot_douzero_f]), ft(N, [bot_douzero_f, bot3_f, bot3_f])

(30, 41)

In [6]:
arena = arn.ARENA(1, True)
arena.registerbot([bot0_f(1), bot0_f(1), bot0_f(1)])
arena.wholegame()

0.3679712 6
0.36502248 3 4 5 6 7 8 9
0.3645744 3 4 5 6 7
0.3546538 6 7 7 7
0.34536824 3 4 5 6 7 8
 0 :  地主 : 6 | 3 4 5 6 7 7 7 7 8 9 10 10 10 J Q Q K A 2
0.6737584 10
0.65769476 K
0.6149257 
0.59642744 9
0.57111585 Q
 1 : 农民1 : 10 | 3 4 5 5 8 8 9 9 Q Q K A A A 小王 大王
0.67901355 
0.6667595 2
0.6274118 K
0.62238467 J
 2 : 农民2 :  | 3 3 4 4 5 6 6 8 9 J J J K K 2 2 2
0.34288463 Q
0.32654926 A
0.31377417 K
0.3091791 
0.297793 J
 3 :  地主 : Q | 3 4 5 6 7 7 7 7 8 9 10 10 10 J Q K A 2
0.66429055 K
0.61128014 
0.52600515 小王
0.5157318 A
0.48422778 大王
 4 : 农民1 : K | 3 4 5 5 8 8 9 9 Q Q A A A 小王 大王
0.6811657 
0.67054224 2
 5 : 农民2 :  | 3 3 4 4 5 6 6 8 9 J J J K K 2 2 2
0.29760423 A
0.28381124 
0.23262507 2
0.027579062 7 7 7 7
 6 :  地主 : A | 3 4 5 6 7 7 7 7 8 9 10 10 10 J Q K 2
0.6479118 
0.56458485 小王
0.505051 大王
0.4275998 小王 大王
 7 : 农民1 :  | 3 4 5 5 8 8 9 9 Q Q A A A 小王 大王
0.6479118 
0.6356567 2
 8 : 农民2 :  | 3 3 4 4 5 6 6 8 9 J J J K K 2 2 2
0.40259123 3 4 5 6 7 8 9 10 J Q K
0.3410499 10 10
0.33645

In [8]:
def bot_mc_f(basebot, verbos=2, RECORD=True, MAXWIDTH=3, NSIMS_FACTOR=50, NSIMS_MAX=1000):
    return mc.BOT(basebot=basebot, verbos=verbos, RECORD=RECORD, MAXWIDTH=MAXWIDTH, NSIMS_FACTOR=NSIMS_FACTOR, NSIMS_MAX=NSIMS_MAX)

In [6]:
import re

def repl(matched):
    d = {"11": "J", "12": "Q", "13": "K", "14": "A", "15": "2", "16": "小王", "17": "大王"}
    return d[matched[0]]

def arenabyscratch(l):
    cards = [rules.str2vec(re.sub(r'1[1-7]', repl, v)) for v in re.split(",", l)]
        
    arena=object.__new__(arn.ARENA)
    arena.verbos = 1
    arena.init = np.array(cards, int)
    arena.remain = arena.init.copy()
    arena.lastplay = np.zeros((3, 15), int)
    arena.pos = 0
    arena.b1 = 2
    arena.b2 = 1
    arena.round = 0
    arena.bot = []
    arena.gameover = False
    arena.winner = None
    arena.getdata()
    arena.RECORD = True
    arena.records = []
    if arena.RECORD:
        arena.records.append(arena.copy())
    return arena

arena = arenabyscratch("3 4 4 4 4 5 5 7 8 9 9 9 10 11 12 13 13 14 14 17, 3 5 5 6 6 6 7 7 7 8 9 10 11 12 13 13 15, 3 3 6 8 8 10 10 11 11 12 12 14 14 15 15 15 16")

In [40]:
model1 = k.models.load_model("model_base2/m{}.keras".format(391350))
model2 = k.models.load_model("model_base2/m{}.keras".format(391300))

In [23]:
w1 = model1.weights[1].numpy().reshape(-1)
w2 = model2.weights[1].numpy().reshape(-1)

In [24]:
w1[0:10], w2[0:10]

(array([-0.00342188, -0.00154246, -0.0020585 , -0.00466309, -0.00450408,
         0.06086061,  0.00401902, -0.00338081,  0.00091003,  0.01286665],
       dtype=float32),
 array([-0.0034434 , -0.00159783, -0.00205496, -0.00465686, -0.00447862,
         0.06076964,  0.0040254 , -0.00332928,  0.00091905,  0.01286706],
       dtype=float32))

In [4]:
model_aug = k.models.load_model("model_aug/m{}.keras".format(0))
bot = bot_aug.BOT(model_aug, verbos=1)

c:\Users\BNCD\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 30 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
arena = arn.ARENA(1, True)
arena.registerbot([bot, bot, bot])

In [6]:
arena.wholegame()

0.507595 3 6 7 7 7 8 8 8 9 9 9 A
0.50668055 3 6 7 7 7 8 8 8 9 9 9 K
0.5066396 6 6
0.50634277 6 6 7 7 7 8 8 8 9 9
0.5062026 3 7 7 7 8 8 8 9 9 9 K A
 0 :  地主 : 3 6 7 7 7 8 8 8 9 9 9 A | 4 6 J Q K K A 2
0.48654562 10 10 10 10
0.48518962 
 1 : 农民1 : 10 10 10 10 | 5 5 5 6 8 9 J J Q Q K A 大王
0.4915114 
 2 : 农民2 :  | 3 3 3 4 4 4 5 6 7 J Q K A 2 2 2 小王
0.5039667 
 3 :  地主 :  | 4 6 J Q K K A 2
0.49775153 5 5 5 Q Q
0.4963311 5 5 5 Q
0.4963237 5 5 5 J
0.49448997 5 5 5 9
0.49381137 Q Q
 4 : 农民1 : 5 5 5 Q Q | 6 8 9 J J K A 大王
0.5029861 3 3 2 2 2
0.5013083 4 4 2 2 2
0.49796736 
 5 : 农民2 : 3 3 2 2 2 | 3 4 4 4 5 6 7 J Q K A 小王
0.49913236 
 6 :  地主 :  | 4 6 J Q K K A 2
0.49756664 
 7 : 农民1 :  | 6 8 9 J J K A 大王
0.50383705 4 4 4 J
0.50320256 3
0.5031651 J
0.50295943 4 4 4 6
0.50279725 3 4 4 4
 8 : 农民2 : 4 4 4 J | 3 5 6 7 Q K A 小王
0.49925733 
 9 :  地主 :  | 4 6 J Q K K A 2
0.50537 
10 : 农民1 :  | 6 8 9 J J K A 大王
0.50698406 5
0.5058 K
0.5057834 3
0.5042151 6
0.50402355 小王
11 : 农民2 : 5 | 3 6 7 Q K A 小王
0.50

In [18]:
a1 = arena.records[3]
# data = bot.getdata(a1)

# 草稿

In [80]:
len(rules.ap)

55632